# TSA Chapter 5: Quiz - Cholesky Ordering

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch5/TSA_ch5_quiz3_cholesky_ordering/TSA_ch5_quiz3_cholesky_ordering.ipynb)

Impact of variable ordering on structural identification.

In [ ]:
!pip install numpy pandas matplotlib statsmodels pandas-datareader -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller, grangercausalitytests, acf
import pandas_datareader.data as web
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}
BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']
plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})
def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

In [ ]:
# Download macro data
try:
    gdp_g = web.DataReader('A191RL1Q225SBEA', 'fred', '2000-01-01', '2024-01-01').resample('QS').last().dropna()
    unemp = web.DataReader('UNRATE', 'fred', '2000-01-01', '2024-01-01').resample('QS').mean().dropna()
    cpi = web.DataReader('CPIAUCSL', 'fred', '2000-01-01', '2024-01-01').resample('QS').last().dropna()
    fed = web.DataReader('FEDFUNDS', 'fred', '2000-01-01', '2024-01-01').resample('QS').mean().dropna()
    macro = pd.DataFrame({'GDP_growth': gdp_g.iloc[:,0], 'Unemployment': unemp.iloc[:,0],
        'Inflation': cpi.iloc[:,0].pct_change(4)*100, 'Fed_rate': fed.iloc[:,0]}).dropna().loc['2001':]
except:
    np.random.seed(42); n = 96; dates = pd.date_range('2001-01-01', periods=n, freq='QS')
    macro = pd.DataFrame({'GDP_growth': 2.5+np.random.normal(0,1.5,n), 'Unemployment': 5.5+1.5*np.sin(np.linspace(0,4*np.pi,n))+np.random.normal(0,0.3,n),
        'Inflation': 2.5+np.random.normal(0,0.8,n), 'Fed_rate': np.maximum(2+2*np.sin(np.linspace(0,3*np.pi,n))+np.random.normal(0,0.2,n),0)}, index=dates)
print(f'Macro data: {len(macro)} obs')

In [ ]:
model = VAR(macro); res = model.fit(2); irf1 = res.irf(20)
macro_rev = macro[macro.columns[::-1]]
res2 = VAR(macro_rev).fit(2); irf2 = res2.irf(20)
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].plot(irf1.orth_irfs[:, 0, 0], color=BLUE, lw=2, label='GDP->GDP')
axes[0].plot(irf1.orth_irfs[:, 1, 0], color=RED, lw=2, label='GDP->Unemp')
axes[0].set_title('Order: GDP first', fontweight='bold'); axes[0].axhline(0, color=GRAY, ls='--')
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False, fontsize=8)
axes[1].plot(irf2.orth_irfs[:, 0, 0], color=ORANGE, lw=2); axes[1].plot(irf2.orth_irfs[:, 1, 0], color=GREEN, lw=2)
axes[1].set_title('Reversed Order', fontweight='bold'); axes[1].axhline(0, color=GRAY, ls='--')
plt.tight_layout(); save_chart(fig, 'ch5_quiz3_cholesky_ordering'); plt.show()